In [102]:
from aimacode.logic import PropKB
from aimacode.planning import Action
from aimacode.search import (
    Node, Problem,
)
from aimacode.utils import expr
from lp_utils import (
    FluentState, encode_state, decode_state,
)
from my_planning_graph import PlanningGraph

from functools import lru_cache


class AirCargoProblem(Problem):
    def __init__(self, cargos, planes, airports, initial: FluentState, goal: list):
        """

        :param cargos: list of str
            cargos in the problem
        :param planes: list of str
            planes in the problem
        :param airports: list of str
            airports in the problem
        :param initial: FluentState object
            positive and negative literal fluents (as expr) describing initial state
        :param goal: list of expr
            literal fluents required for goal test
        """
        self.state_map = initial.pos + initial.neg
        self.initial_state_TF = encode_state(initial, self.state_map)
        Problem.__init__(self, self.initial_state_TF, goal=goal)
        self.cargos = cargos
        self.planes = planes
        self.airports = airports
        self.actions_list = self.get_actions()

        #--- additional custom helper variables
        #--- state_to_action is a dict that maps the state_TF to action
        #--- it is built when actions is called

    def get_actions(self):
        """
        This method creates concrete actions (no variables) for all actions in the problem
        domain action schema and turns them into complete Action objects as defined in the
        aimacode.planning module. It is computationally expensive to call this method directly;
        however, it is called in the constructor and the results cached in the `actions_list` property.

        Returns:
        ----------
        list<Action>
            list of Action objects
        """

        # TODO create concrete Action objects based on the domain action schema for: Load, Unload, and Fly
        # concrete actions definition: specific literal action that does not include variables as with the schema
        # for example, the action schema 'Load(c, p, a)' can represent the concrete actions 'Load(C1, P1, SFO)'
        # or 'Load(C2, P2, JFK)'.  The actions for the planning problem must be concrete because the problems in
        # forward search and Planning Graphs must use Propositional Logic

        def load_actions():
            """Create all concrete Load actions and return a list

            :return: list of Action objects
            """
            loads = []
            # TODO create all load ground actions from the domain Load action
            for c in self.cargos:
                for  p in self.planes:
                    for a in self.airports:
                        #--- preconds:At(P, A) and At(C, A)
                        precond_pos = [expr_at(p, a), expr_at(c, a)]
                        precond_neg = []
                        effect_add = [expr_in(c, p)]
                        effect_rem = [expr_at(c, a)]
                        load = Action (expr_load(c, p, a), 
                                       [precond_pos, precond_neg],
                                       [effect_add, effect_rem])
                        loads.append(load)
            return loads

        def unload_actions():
            """Create all concrete Unload actions and return a listcond_neg = []

            :return: list of Action objects
            """
            unloads = []
            # TODO create all Unload ground actions from the domain Unload action
            for c in self.cargos:
                for p in self.planes:
                    for a in self.airports:
                        #--- preconds:At(P, A) and At(C, A)
                        precond_pos = [expr_at(p, a), expr_in(c, p)]
                        precond_neg = []
                        effect_add = [expr_at(c, a)]
                        effect_rem = [expr_in(c, p)]
                        unload = Action (expr_unload(c, p, a), 
                                       [precond_pos, precond_neg],
                                       [effect_add, effect_rem])
                        unloads.append(unload)
            return unloads

        def fly_actions():
            """Create all concrete Fly actions and return a list

            :return: list of Action objects
            """
            flys = []
            for fr in self.airports:
                for to in self.airports:
                    if fr != to:
                        for p in self.planes:
                            precond_pos = [expr("At({}, {})".format(p, fr))]
                            precond_neg = []
                            effect_add = [expr("At({}, {})".format(p, to))]
                            effect_rem = [expr("At({}, {})".format(p, fr))]
                            fly = Action(expr("Fly({}, {}, {})".format(p, fr, to)),
                                         [precond_pos, precond_neg],
                                         [effect_add, effect_rem])
                            flys.append(fly)
            return flys

        return load_actions() + unload_actions() + fly_actions()

    def actions(self, state: str) -> list:
        """ Return the actions that can be executed in the given state.

        :param state: str
            state represented as T/F string of mapped fluents (state variables)
            e.g. 'FTTTFF'
        :return: list of Action objects
        """
        # TODO implement
        possible_actions = []
        kb = PropKB()
        kb.tell(decode_state(state, self.state_map).pos_sentence())
        for action in self.actions_list:
            if action.check_precond(kb, action.args):
                possible_actions.append(action)
           
        return possible_actions

    def result(self, state: str, action: Action):
        """ Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state).

        :param state: state entering node
        :param action: Action applied
        :return: resulting state after action
        """
        # TODO implement
        new_state = FluentState([], [])
        kb = PropKB()
        kb.tell(decode_state(state, self.state_map).pos_sentence())

        action(kb, action.args)

        for clause in kb.clauses:
            new_state.pos.append(clause)
        for clause in action.effect_rem:
            new_state.neg.append(clause)
        
        return encode_state(new_state, self.state_map)

    def goal_test(self, state: str) -> bool:
        """ Test the state to see if goal is reached

        :param state: str representing state
        :return: bool
        """
        kb = PropKB()
        kb.tell(decode_state(state, self.state_map).pos_sentence())
        for clause in self.goal:
            if clause not in kb.clauses:
                return False
        return True

    def h_1(self, node: Node):
        # note that this is not a true heuristic
        h_const = 1
        return h_const

    @lru_cache(maxsize=8192)
    def h_pg_levelsum(self, node: Node):
        """This heuristic uses a planning graph representation of the problem
        state space to estimate the sum of all actions that must be carried
        out from the current state in order to satisfy each individual goal
        condition.
        """
        # requires implemented PlanningGraph class
        pg = PlanningGraph(self, node.state)
        pg_levelsum = pg.h_levelsum()
        return pg_levelsum

    @lru_cache(maxsize=8192)
    def h_ignore_preconditions(self, node: Node):
        """This heuristic estimates the minimum number of actions that must be
        carried out from the current state in order to satisfy all of the goal
        conditions by ignoring the preconditions required for an action to be
        executed.
        """
        # TODO implement (see Russell-Norvig Ed-3 10.2.3  or Russell-Norvig Ed-2 11.2)
        
        #--- the number of steps required to solve a conjunction of goals is the number of unsatisfied goals
        count = 0
        state = decode_state(node.state, self.state_map).pos
        for clause in self.goal:
            if not clause in state:
                #--- the goal clause is not satisfied
                #--- therefore, there is at least one action that needs to be taken
                #--- so count that action
                count +=1
        
        return count


def air_cargo_p1() -> AirCargoProblem:
    cargos = ['C1', 'C2']
    planes = ['P1', 'P2']
    airports = ['JFK', 'SFO']
    pos = [expr('At(C1, SFO)'),
           expr('At(C2, JFK)'),
           expr('At(P1, SFO)'),
           expr('At(P2, JFK)'),
           ]
    neg = [expr('At(C2, SFO)'),
           expr('In(C2, P1)'),
           expr('In(C2, P2)'),
           expr('At(C1, JFK)'),
           expr('In(C1, P1)'),
           expr('In(C1, P2)'),
           expr('At(P1, JFK)'),
           expr('At(P2, SFO)'),
           ]
    init = FluentState(pos, neg)
    goal = [expr('At(C1, JFK)'),
            expr('At(C2, SFO)'),
            ]
    return AirCargoProblem(cargos, planes, airports, init, goal)


def air_cargo_p2() -> AirCargoProblem:
    # TODO implement Problem 2 definition
    cargos = ['C1', 'C2', 'C3']
    planes = ['P1', 'P2', 'P3']
    airports = ['JFK', 'SFO', 'ATL']
    pos = [expr('At(C1, SFO)'),
           expr('At(C2, JFK)'),
           expr('At(C3, ATL)'),
           expr('At(P1, SFO)'),
           expr('At(P2, JFK)'),
           expr('At(P3, ATL)')
           ]
    neg = [expr('At(C2, SFO)'),
           expr('At(C2, ATL)'),           
           expr('In(C2, P1)'),
           expr('In(C2, P2)'),
           expr('In(C2, P3)'),
           expr('At(C1, JFK)'),
           expr('At(C1, ATL)'),
           expr('In(C1, P1)'),
           expr('In(C1, P2)'),
           expr('In(C1, P3)'),
           expr('At(C3, SFO)'),
           expr('At(C3, JFK)'),
           expr('In(C3, P1)'),
           expr('In(C3, P2)'),
           expr('In(C3, P3)'),
           expr('At(P1, JFK)'),
           expr('At(P1, ATL)'),
           expr('At(P2, SFO)'),
           expr('At(P2, ATL)'),           
           expr('At(P3, SFO)'),                      
           expr('At(P3, JFK)'),                                 
           ]
    init = FluentState(pos, neg)
    goal = [expr('At(C1, JFK)'),
            expr('At(C2, SFO)'),
            expr('At(C3, SFO)'),
            ]
    return AirCargoProblem(cargos, planes, airports, init, goal)


def air_cargo_p3() -> AirCargoProblem:
    cargos = ['C1', 'C2', 'C3', 'C4']
    planes = ['P1', 'P2']
    airports = ['JFK', 'SFO', 'ATL', 'ORD']
    pos = [expr('At(C1, SFO)'),
           expr('At(C2, JFK)'),
           expr('At(C3, ATL)'),
           expr('At(C4, ORD)'),
           expr('At(P1, SFO)'),
           expr('At(P2, JFK)'),
           ]
    neg = [expr('At(C2, SFO)'),
           expr('At(C2, ATL)'),           
           expr('At(C2, ORD)'),           
           expr('In(C2, P1)'),
           expr('In(C2, P2)'),
           expr('At(C1, JFK)'),
           expr('At(C1, ATL)'),
           expr('At(C1, ORD)'),
           expr('In(C1, P1)'),
           expr('In(C1, P2)'),
           expr('At(C3, JFK)'),
           expr('At(C3, SFO)'),
           expr('At(C3, ORD)'),
           expr('In(C3, P1)'),
           expr('In(C3, P2)'),
           expr('At(C4, JFK)'),
           expr('At(C4, SFO)'),
           expr('At(C4, ATL)'),
           expr('In(C4, P1)'),
           expr('In(C4, P2)'),
           #--- since plane 1 is at SFO, it is not at JFK, ATL, or ORD           
           expr('At(P1, JFK)'),
           expr('At(P1, ATL)'),
           expr('At(P1, ORD)'),
           #--- since plane 2 is at JFK, it is not at SFO, ATL, or ORD
           expr('At(P2, SFO)'),
           expr('At(P2, ATL)'),           
           expr('At(P2, ORD)'),           
           ]
    init = FluentState(pos, neg)
    goal = [expr('At(C1, JFK)'),
            expr('At(C2, SFO)'),
            expr('At(C3, JFK)'),
            expr('At(C4, SFO)'),
            ]
    return AirCargoProblem(cargos, planes, airports, init, goal)

#----------------------------
# helper functions

def expr_op(op, *args):
    if len(args) == 2:
        return expr("{}({}, {})".format(op, args[0], args[1]))
    else:
        return expr("{}({}, {}, {})".format(op, args[0], args[1], args[2]))

def expr_at (obj, airport):
    return expr_op("At", obj, airport)

def expr_in (cargo, plane):
    return expr_op("In", cargo, plane)

def expr_load (cargo, plane, airport):
    return expr_op("Load", cargo, plane, airport)
    
def expr_unload (cargo, plane, airport):
    return expr_op("Unload", cargo, plane, airport)



In [44]:
def sop (op, *args):
    print(args)
    if len(args) == 2:
        return op + "({}, {})".format(args[0], args[1])
    else:
        return op + "({}, {}, {})".format(args[0], args[1], args[2])
    
    
print(sop ("In", "C1", "P1"))

('C1', 'P1')
In(C1, P1)


In [13]:
p = air_cargo_p2()

print (p.get_actions())
print (p.initial_state_TF)
print (p.state_map)
fs = decode_state (p.initial_state_TF, p.state_map)
print ("State:", fs.pos)

possible_actions = []
for a in p.actions_list:
    if set(a.precond_pos) < set(fs.pos):
        possible_actions.append(a)
        
print("possible actions")
for a in p.actions(p.initial_state_TF):
    print (a.name, a.args)
    
print("possible actions")
for a in possible_actions:
    print (a.name, a.args)

print("possible actions")
for a in p.actions(p.initial_state_TF):
    print (a.name, a.args)
    
r = p.result (p.initial_state_TF, possible_actions[0])
print (r)
fs = decode_state(r, p.state_map)
print ("State:", fs.pos)
print ("State:", fs.neg)

[<aimacode.planning.Action object at 0x00000239F54F9438>, <aimacode.planning.Action object at 0x00000239F54F97B8>, <aimacode.planning.Action object at 0x00000239F54F9B38>, <aimacode.planning.Action object at 0x00000239F54F9EB8>, <aimacode.planning.Action object at 0x00000239F54C45F8>, <aimacode.planning.Action object at 0x00000239F54C4D30>, <aimacode.planning.Action object at 0x00000239F54C4EB8>, <aimacode.planning.Action object at 0x00000239F54B84A8>, <aimacode.planning.Action object at 0x00000239F54B8CC0>, <aimacode.planning.Action object at 0x00000239F54AF4A8>, <aimacode.planning.Action object at 0x00000239F54AF6D8>, <aimacode.planning.Action object at 0x00000239F54AF908>, <aimacode.planning.Action object at 0x00000239F54AFDA0>, <aimacode.planning.Action object at 0x00000239F54BA978>, <aimacode.planning.Action object at 0x00000239F54BA400>, <aimacode.planning.Action object at 0x00000239F5493240>, <aimacode.planning.Action object at 0x00000239F5493470>, <aimacode.planning.Action obje

In [103]:
from run_search import run_search
from aimacode.search import (breadth_first_search, astar_search,
    breadth_first_tree_search, depth_first_graph_search, uniform_cost_search,
    greedy_best_first_graph_search, depth_limited_search,
    recursive_best_first_search)


p = air_cargo_p3()
s = breadth_first_search
print ("breadth first search:")
run_search (p, s)


p = air_cargo_p3()
s = astar_search
print ("astar search, ignore:")
run_search (p, s, p.h_ignore_preconditions)

p = air_cargo_p3()
s = astar_search
print ("astar search, h_1:")
run_search (p, s, p.h_1)




breadth first search:

Expansions   Goal Tests   New Nodes
  14663       18098       129631  

Plan length: 12  Time elapsed in seconds: 126.23463102133246
Load(C1, P1, SFO)
Load(C2, P2, JFK)
Fly(P2, JFK, ORD)
Load(C4, P2, ORD)
Fly(P1, SFO, ATL)
Load(C3, P1, ATL)
Fly(P1, ATL, JFK)
Unload(C1, P1, JFK)
Unload(C3, P1, JFK)
Fly(P2, ORD, SFO)
Unload(C2, P2, SFO)
Unload(C4, P2, SFO)

astar search, ignore:

Expansions   Goal Tests   New Nodes
   5040        5042       44944   

Plan length: 12  Time elapsed in seconds: 26.508035168677452
Load(C2, P2, JFK)
Fly(P2, JFK, ORD)
Load(C4, P2, ORD)
Fly(P2, ORD, SFO)
Unload(C4, P2, SFO)
Load(C1, P1, SFO)
Fly(P1, SFO, ATL)
Load(C3, P1, ATL)
Fly(P1, ATL, JFK)
Unload(C3, P1, JFK)
Unload(C1, P1, JFK)
Unload(C2, P2, SFO)

astar search, h_1:

Expansions   Goal Tests   New Nodes
  18235       18237       159716  

Plan length: 12  Time elapsed in seconds: 93.71817021203606
Load(C1, P1, SFO)
Load(C2, P2, JFK)
Fly(P1, SFO, ATL)
Load(C3, P1, ATL)
Fly(P2, JFK, O